# Predicting Sales - Data Preparation

To do:
 * don't include any rebalanced wins in test/validation data
 * Range validation, throw exceptions

https://www.geeksforgeeks.org/stratified-sampling-in-pandas/

In [64]:
# Define some exclusions for PEP8 that don't apply when the Jupyter Notebook
#   is exported to .py file
# pylint: disable=pointless-statement
# pylint: disable=fixme
# pylint: disable=expression-not-assigned
# pylint: disable=missing-module-docstring
# pylint: disable=invalid-name

import os
import importlib

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


import saleslib # my custom one
importlib.reload(saleslib)

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999
sns.set_theme(style="darkgrid")

# Optional performance enhancements, don't always work in Lambda Labs instances
# %load_ext Cython
# import numba
# numba.set_num_threads(4)

In [65]:
input_filepath = os.path.join('data', 'raw_CRM_opps_export-dummydata.csv')
df = pd.read_csv(input_filepath)
df.head()

,Age of opp in days,CRM Identifier,Deal size (USD),Industry,Num times opp pushed,Sales Rep ID,Sales team name,Won,array_of_sfdc_formulas
0,502.0,opp_4f70539,82500.0,Technology,1,rep_9c52b45,Central - South 1,0,"{ ""quarter_created"": 2, ""quarter_closed"": 4, ""..."
1,214.0,opp_3ff4389,5000.0,Finance,2,rep_9c52b45,Central - South 1,0,"{ ""quarter_created"": 1, ""quarter_closed"": 4, ""..."
2,262.0,opp_2819115,12736.0,Finance,3,rep_9c52b45,Central - South 1,1,"{ ""quarter_created"": 1, ""quarter_closed"": 4, ""..."
3,69.0,opp_72dcc7b,57867.2,Technology,1,rep_9c52b45,Central - North,1,"{ ""quarter_created"": 3, ""quarter_closed"": 4, ""..."
4,134.0,opp_08a2884,30000.0,Finance,4,rep_9c52b45,Central - South 1,0,"{ ""quarter_created"": 2, ""quarter_closed"": 4, ""..."


# Pipelines

In [66]:
def start_pipeline(df):
    return df.copy()

#print(df.shape)

df_phase1 = (df.
                pipe(start_pipeline).
                pipe(saleslib.convert_json_to_features).
                pipe(saleslib.set_datatypes).
                pipe(saleslib.drop_unused_columns).
                pipe(saleslib.add_features_sales_type)
)

orig_num_rows = df.shape[0]

df_phase1.describe()

[Set Datatypes] Recast datatypes
[Drop Unused Columns] Dropped -1 columns


,Age of opp in days,Deal size (USD),Num times opp pushed,Won,quarter_created,quarter_closed,partner_involved
count,9463.0,9463.000000,9463.0,9463.0,9463.0,9463.0,9463.0
mean,126.369862,44169.443940,1.824157,0.261545,2.436225,2.512417,0.371552
std,134.746506,44604.121307,1.93012,0.439499,1.107098,1.139419,0.483245
min,0.0,0.000000,0.0,0.0,1.0,1.0,0.0
25%,37.0,11264.000000,0.0,0.0,1.0,1.0,0.0
50%,84.0,30000.000000,1.0,0.0,2.0,2.0,0.0
75%,168.0,65000.000000,3.0,1.0,3.0,4.0,1.0
max,1463.0,424287.000000,16.0,1.0,4.0,4.0,1.0


In [67]:
df_phase1.head(20)

print(df_phase1['positioning_category'].unique())
print(df_phase1['product_family'].unique())
print(df_phase1['hosting_location'].unique())

['product' 'managed_service' 'limited_service_engagement' None 'unclear']
['C' 'B' 'D' 'A' 'W' 'E' 'Q' None 'F']
['saas_platform' '' 'on_prem' None 'legacy_acquisition']


In [68]:
df_phase1[df_phase1['hosting_location']== '']

,Age of opp in days,Deal size (USD),Industry,Num times opp pushed,Sales Rep ID,Sales team name,Won,quarter_created,quarter_closed,partner_involved,positioning_category,product_family,hosting_location
15,343,12000,Healthcare,6,rep_9c52b45,Central - North,0,1,1,1,managed_service,C,
20,126,30000,<NA>,0,rep_9c52b45,Central - North,0,4,1,0,managed_service,A,
24,0,259606,Finance,0,rep_9c52b45,Central - South 1,1,1,1,1,limited_service_engagement,W,
26,154,60000,Services,2,rep_9c52b45,Central - North,0,4,2,0,managed_service,C,
28,112,20000,Manufacturing,1,rep_9c52b45,Central - North,0,1,2,1,managed_service,C,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,209,89000,Manufacturing,3,rep_a04127f,West - Rockies,0,4,2,1,managed_service,C,
9444,30,30000,Services,3,rep_e390d7d,East - North,0,3,4,1,managed_service,A,
9445,96,70125,Technology,4,rep_e390d7d,East - North,1,3,4,1,managed_service,C,
9449,257,15000,Finance,3,rep_e390d7d,East - North,0,1,4,0,managed_service,B,


In [69]:
df_phase2 = (df_phase1.
                pipe(saleslib.drop_rows_outside_ranges).
                pipe(saleslib.drop_rows_missing_values_in_columns)
)
df_phase2.describe()

[Drop Rows outside range] Dropping 867 rows (9.16%) from Age of opp in days because value was < 14
[Drop Rows outside range] Dropping 46 rows (0.49%) from Deal size (USD) because value was < 1500
[Drop Rows outside range] Dropping 64 rows (0.68%) from Age of opp in days because value was > 720
[Drop Rows outside range] Dropped a total of 977 rows (10.32%) because values were outside ranges


,Age of opp in days,Deal size (USD),Num times opp pushed,Won,quarter_created,quarter_closed,partner_involved
count,8486.0,8486.000000,8486.0,8486.0,8486.0,8486.0,8486.0
mean,133.168277,46014.863186,1.952864,0.234268,2.430592,2.514612,0.377681
std,119.560638,44730.531910,1.893298,0.423566,1.101914,1.135265,0.484836
min,14.0,1500.000000,0.0,0.0,1.0,1.0,0.0
25%,48.0,12375.000000,1.0,0.0,1.0,1.0,0.0
50%,96.0,31468.000000,1.0,0.0,2.0,2.0,0.0
75%,178.0,66000.000000,3.0,0.0,3.0,4.0,1.0
max,720.0,424287.000000,14.0,1.0,4.0,4.0,1.0


In [70]:
df_phase3 = (df_phase2.
                pipe(saleslib.winsorize_cols)
)
df_phase3.describe()

[Winsorized Low End] Winsorized 1030 rows (12.14%) in feature "Age of opp in days" because value was < 30
[Winsorized Low End] Winsorized 1453 rows (17.12%) in feature "Deal size (USD)" because value was < 10000
[Winsorized High End] Winsorized 478 rows (5.63%) in feature "Age of opp in days" because value was > 365
[Winsorized High End] Winsorized 164 rows (1.93%) in feature "Deal size (USD)" because value was > 180000
[Winsorized High End] Winsorized 269 rows (3.17%) in feature "Num times opp pushed" because value was > 6


,Age of opp in days,Deal size (USD),Num times opp pushed,Won,quarter_created,quarter_closed,partner_involved
count,8486.0,8486.000000,8486.0,8486.0,8486.0,8486.0,8486.0
mean,127.51367,45745.887344,1.885812,0.234268,2.430592,2.514612,0.377681
std,98.470925,40215.369943,1.683724,0.423566,1.101914,1.135265,0.484836
min,30.0,10000.000000,0.0,0.0,1.0,1.0,0.0
25%,48.0,12375.000000,1.0,0.0,1.0,1.0,0.0
50%,96.0,31468.000000,1.0,0.0,2.0,2.0,0.0
75%,178.0,66000.000000,3.0,0.0,3.0,4.0,1.0
max,365.0,180000.000000,6.0,1.0,4.0,4.0,1.0


In [71]:
df_phase4 = (df_phase3.
                pipe(saleslib.normalize_cols_to_fixed_range).
                pipe(saleslib.add_feature_sales_team)
)
df_phase4.describe()

[Normalized Absolute] Normalized range of feature "Age of opp in days" from [30,365] to [0.0,1.0] using absolute range of [30,365]
[Normalized Absolute] Normalized range of feature "Deal size (USD)" from [10000,180000] to [0.0,1.0] using absolute range of [10000,180000]
[Normalized Absolute] Normalized range of feature "Num times opp pushed" from [0,6] to [0.0,1.0] using absolute range of [0,6]
[Normalized Absolute] Normalized range of feature "quarter_created" from [1,4] to [0.0,1.0] using absolute range of [1,4]
[Normalized Absolute] Normalized range of feature "quarter_closed" from [1,4] to [0.0,1.0] using absolute range of [1,4]


,Age of opp in days,Deal size (USD),Num times opp pushed,Won,quarter_created,quarter_closed,partner_involved
count,8486.0,8486.000000,8486.0,8486.0,8486.0,8486.0,8486.0
mean,0.291086,0.210270,0.314302,0.234268,0.476864,0.504871,0.377681
std,0.293943,0.236561,0.280621,0.423566,0.367305,0.378422,0.484836
min,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.053731,0.013971,0.166667,0.0,0.0,0.0,0.0
50%,0.197015,0.126282,0.166667,0.0,0.333333,0.333333,0.0
75%,0.441791,0.329412,0.5,0.0,0.666667,1.0,1.0
max,1.0,1.000000,1.0,1.0,1.0,1.0,1.0


In [72]:
# # view unique values for string columns:
# import json
# xyz = saleslib.list_unique_string_values(df_phase4)
# print(xyz)

In [73]:
df_phase4.head(20)

,Age of opp in days,Deal size (USD),Industry,Num times opp pushed,Sales Rep ID,Sales team name,Won,quarter_created,quarter_closed,partner_involved,positioning_category,product_family,hosting_location,sales_territory
0,1.0,0.426471,Technology,0.166667,rep_9c52b45,Central - South 1,0,0.333333,1.0,0,product,C,saas_platform,Central
1,0.549254,0.000000,Finance,0.333333,rep_9c52b45,Central - South 1,0,0.0,1.0,1,product,B,saas_platform,Central
2,0.692537,0.016094,Finance,0.5,rep_9c52b45,Central - South 1,1,0.0,1.0,0,product,B,saas_platform,Central
3,0.116418,0.281571,Technology,0.166667,rep_9c52b45,Central - North,1,0.666667,1.0,1,product,C,saas_platform,Central
4,0.310448,0.117647,Finance,0.666667,rep_9c52b45,Central - South 1,0,0.333333,1.0,1,product,B,saas_platform,Central
6,0.026866,0.088235,Utilities & Energy,0.0,rep_9c52b45,Central - South 1,0,1.0,1.0,1,product,C,saas_platform,Central
7,1.0,0.294118,Finance,0.5,rep_9c52b45,Central - North,0,0.333333,1.0,1,product,B,saas_platform,Central
8,0.035821,0.007859,Finance,0.166667,rep_9c52b45,Central - South 1,1,1.0,1.0,0,product,B,saas_platform,Central
9,0.447761,0.058824,Finance,0.833333,rep_9c52b45,Central - South 1,0,0.333333,1.0,1,product,B,saas_platform,Central
11,1.0,0.007435,Finance,0.833333,rep_9c52b45,Central - North,0,0.0,0.0,0,product,B,saas_platform,Central


In [74]:
df_phase5 = (df_phase4.
                pipe(saleslib.onehotencode_string_columns).
                pipe(saleslib.convert_all_boolean_cols_to_int).
                pipe(saleslib.upsample_wins).                       # near the end
                pipe(saleslib.rename_features).                     # next to last
                pipe(saleslib.assert_datatypes_ready_for_training)  # should prob be near the end
)
df_phase5.describe()

[One Hot Encode] Encoded feature "Industry", added 13 new columns. Dropped original.
[One Hot Encode] Encoded feature "Sales Rep ID", added 271 new columns. Dropped original.
[One Hot Encode] Encoded feature "Sales team name", added 13 new columns. Dropped original.
[One Hot Encode] Encoded feature "positioning_category", added 4 new columns. Dropped original.
[One Hot Encode] Encoded feature "product_family", added 8 new columns. Dropped original.
[One Hot Encode] Encoded feature "hosting_location", added 4 new columns. Dropped original.
[One Hot Encode] Encoded feature "sales_territory", added 3 new columns. Dropped original.
[Convert] Converted 316 columns from Boolean to uint8[pyarrow] for XGBoost
[WinLossInfo] Wins: 1988 23.4%  Losses: 6498  Total: 8486
[Upsampling Wins] Resampling wins from 1988 opps to 3180 opps...
[WinLossInfo] Wins: 3180 32.9%  Losses: 6498  Total: 9678
[Rename Features] Renamed column names


,age,revenue,pushes,Won,quarter_created,quarter_closed,partner,Industry_Communications & Media,Industry_Education,Industry_Finance,Industry_Government,Industry_Healthcare,Industry_Manufacturing,Industry_Other Industry,Industry_Real Estate,Industry_Retail,Industry_Services,Industry_Technology,Industry_Transportation,Industry_Utilities & Energy,Sales Rep ID_rep_013a112,Sales Rep ID_rep_037508f,Sales Rep ID_rep_04069ca,Sales Rep ID_rep_05fbad0,Sales Rep ID_rep_06c9522,Sales Rep ID_rep_071e409,Sales Rep ID_rep_074d4cc,Sales Rep ID_rep_0821b39,Sales Rep ID_rep_0a05cf9,Sales Rep ID_rep_0bd3379,Sales Rep ID_rep_0bfa997,Sales Rep ID_rep_0d79d17,Sales Rep ID_rep_0f46ca3,Sales Rep ID_rep_103e9e7,Sales Rep ID_rep_114b3c7,Sales Rep ID_rep_1199816,Sales Rep ID_rep_11aa01a,Sales Rep ID_rep_1221463,Sales Rep ID_rep_12c8fa0,Sales Rep ID_rep_12d263e,Sales Rep ID_rep_1366a31,Sales Rep ID_rep_136ab1d,Sales Rep ID_rep_139be21,Sales Rep ID_rep_13eec1f,Sales Rep ID_rep_1911baf,Sales Rep ID_rep_198b0be,Sales Rep ID_rep_1a362bd,Sales Rep ID_rep_1c0c915,Sales Rep ID_rep_1d0c578,Sales Rep ID_rep_1dbb64c,Sales Rep ID_rep_1e1b6b1,Sales Rep ID_rep_1ef511c,Sales Rep ID_rep_1f7feea,Sales Rep ID_rep_201e738,Sales Rep ID_rep_20c37f1,Sales Rep ID_rep_228133a,Sales Rep ID_rep_22d1012,Sales Rep ID_rep_2315b4b,Sales Rep ID_rep_23a1a55,Sales Rep ID_rep_23d0543,Sales Rep ID_rep_23ddec5,Sales Rep ID_rep_247be15,Sales Rep ID_rep_24bd6fd,Sales Rep ID_rep_263e0f2,Sales Rep ID_rep_27750b6,Sales Rep ID_rep_27b8985,Sales Rep ID_rep_289a13f,Sales Rep ID_rep_299b3af,Sales Rep ID_rep_29b5303,Sales Rep ID_rep_29e08a6,Sales Rep ID_rep_2a946da,Sales Rep ID_rep_2bdb202,Sales Rep ID_rep_2d76014,Sales Rep ID_rep_2e3de83,Sales Rep ID_rep_2e817b7,Sales Rep ID_rep_2e9de07,Sales Rep ID_rep_2fef61b,Sales Rep ID_rep_30e692c,Sales Rep ID_rep_3381ce0,Sales Rep ID_rep_339dd94,Sales Rep ID_rep_33d8e5f,Sales Rep ID_rep_3427e42,Sales Rep ID_rep_34a27e3,Sales Rep ID_rep_3582291,Sales Rep ID_rep_36c0074,Sales Rep ID_rep_373f7d0,Sales Rep ID_rep_388ecf9,Sales Rep ID_rep_38dd901,Sales Rep ID_rep_39f4987,Sales Rep ID_rep_3c02d74,Sales Rep ID_rep_3c17971,Sales Rep ID_rep_3eec4d1,Sales Rep ID_rep_4324a3d,Sales Rep ID_rep_442171a,Sales Rep ID_rep_4477cba,Sales Rep ID_rep_452a29c,Sales Rep ID_rep_45f5ba6,Sales Rep ID_rep_469891d,Sales Rep ID_rep_47b7e08,Sales Rep ID_rep_488c4bf,Sales Rep ID_rep_493273f,Sales Rep ID_rep_49d39a4,Sales Rep ID_rep_4a7474a,Sales Rep ID_rep_4c1407b,Sales Rep ID_rep_4cac043,Sales Rep ID_rep_4d5616b,Sales Rep ID_rep_4fbad23,Sales Rep ID_rep_4fcd9d8,Sales Rep ID_rep_523f514,Sales Rep ID_rep_52a8010,Sales Rep ID_rep_52c3eb3,Sales Rep ID_rep_536a60e,Sales Rep ID_rep_53c4626,Sales Rep ID_rep_567e972,Sales Rep ID_rep_584d289,Sales Rep ID_rep_58988cb,Sales Rep ID_rep_58f4498,Sales Rep ID_rep_5ad3489,Sales Rep ID_rep_5af5623,Sales Rep ID_rep_5b975b2,Sales Rep ID_rep_5bdd867,Sales Rep ID_rep_5bed9c7,Sales Rep ID_rep_5c8b2b8,Sales Rep ID_rep_5d21642,Sales Rep ID_rep_5d420f5,Sales Rep ID_rep_5f9b0b5,Sales Rep ID_rep_620efdc,Sales Rep ID_rep_627b9bb,Sales Rep ID_rep_6372974,Sales Rep ID_rep_639f828,Sales Rep ID_rep_652d327,Sales Rep ID_rep_6705ebb,Sales Rep ID_rep_684c2e1,Sales Rep ID_rep_6873b3c,Sales Rep ID_rep_6951398,Sales Rep ID_rep_69c4542,Sales Rep ID_rep_6a69c35,Sales Rep ID_rep_6a817aa,Sales Rep ID_rep_6a87d28,Sales Rep ID_rep_6ad6615,Sales Rep ID_rep_6cdce0d,Sales Rep ID_rep_6e553b7,Sales Rep ID_rep_6f2b653,Sales Rep ID_rep_6fc14f7,Sales Rep ID_rep_70630bf,Sales Rep ID_rep_7084626,Sales Rep ID_rep_7089803,Sales Rep ID_rep_71cd7dc,Sales Rep ID_rep_720c136,Sales Rep ID_rep_72c1303,Sales Rep ID_rep_74a4d9c,Sales Rep ID_rep_74edb95,Sales Rep ID_rep_74f24b9,Sales Rep ID_rep_76f5957,Sales Rep ID_rep_7740707,Sales Rep ID_rep_785b538,Sales Rep ID_rep_7b00993,Sales Rep ID_rep_7bc1730,Sales Rep ID_rep_7c5f2d7,Sales Rep ID_rep_7e853a4,Sales Rep ID_rep_82bc742,Sales Rep ID_rep_83cfd47,Sales Rep ID_rep_83ec8cf,Sales Rep ID_rep_852d43c,Sales Rep ID_rep_85503ea,Sales Rep ID_rep_85910c4,Sales

In [75]:
final_num_rows = df_phase5.shape[0]
percent_row_change = 100 * (final_num_rows - orig_num_rows) / orig_num_rows
print(f'Num rows went from {orig_num_rows} to {final_num_rows}  {percent_row_change:0.2f}% change')

Num rows went from 9463 to 9678  2.27% change


In [76]:
df_phase5.head(3)

,age,revenue,pushes,Won,quarter_created,quarter_closed,partner,Industry_Communications & Media,Industry_Education,Industry_Finance,Industry_Government,Industry_Healthcare,Industry_Manufacturing,Industry_Other Industry,Industry_Real Estate,Industry_Retail,Industry_Services,Industry_Technology,Industry_Transportation,Industry_Utilities & Energy,Sales Rep ID_rep_013a112,Sales Rep ID_rep_037508f,Sales Rep ID_rep_04069ca,Sales Rep ID_rep_05fbad0,Sales Rep ID_rep_06c9522,Sales Rep ID_rep_071e409,Sales Rep ID_rep_074d4cc,Sales Rep ID_rep_0821b39,Sales Rep ID_rep_0a05cf9,Sales Rep ID_rep_0bd3379,Sales Rep ID_rep_0bfa997,Sales Rep ID_rep_0d79d17,Sales Rep ID_rep_0f46ca3,Sales Rep ID_rep_103e9e7,Sales Rep ID_rep_114b3c7,Sales Rep ID_rep_1199816,Sales Rep ID_rep_11aa01a,Sales Rep ID_rep_1221463,Sales Rep ID_rep_12c8fa0,Sales Rep ID_rep_12d263e,Sales Rep ID_rep_1366a31,Sales Rep ID_rep_136ab1d,Sales Rep ID_rep_139be21,Sales Rep ID_rep_13eec1f,Sales Rep ID_rep_1911baf,Sales Rep ID_rep_198b0be,Sales Rep ID_rep_1a362bd,Sales Rep ID_rep_1c0c915,Sales Rep ID_rep_1d0c578,Sales Rep ID_rep_1dbb64c,Sales Rep ID_rep_1e1b6b1,Sales Rep ID_rep_1ef511c,Sales Rep ID_rep_1f7feea,Sales Rep ID_rep_201e738,Sales Rep ID_rep_20c37f1,Sales Rep ID_rep_228133a,Sales Rep ID_rep_22d1012,Sales Rep ID_rep_2315b4b,Sales Rep ID_rep_23a1a55,Sales Rep ID_rep_23d0543,Sales Rep ID_rep_23ddec5,Sales Rep ID_rep_247be15,Sales Rep ID_rep_24bd6fd,Sales Rep ID_rep_263e0f2,Sales Rep ID_rep_27750b6,Sales Rep ID_rep_27b8985,Sales Rep ID_rep_289a13f,Sales Rep ID_rep_299b3af,Sales Rep ID_rep_29b5303,Sales Rep ID_rep_29e08a6,Sales Rep ID_rep_2a946da,Sales Rep ID_rep_2bdb202,Sales Rep ID_rep_2d76014,Sales Rep ID_rep_2e3de83,Sales Rep ID_rep_2e817b7,Sales Rep ID_rep_2e9de07,Sales Rep ID_rep_2fef61b,Sales Rep ID_rep_30e692c,Sales Rep ID_rep_3381ce0,Sales Rep ID_rep_339dd94,Sales Rep ID_rep_33d8e5f,Sales Rep ID_rep_3427e42,Sales Rep ID_rep_34a27e3,Sales Rep ID_rep_3582291,Sales Rep ID_rep_36c0074,Sales Rep ID_rep_373f7d0,Sales Rep ID_rep_388ecf9,Sales Rep ID_rep_38dd901,Sales Rep ID_rep_39f4987,Sales Rep ID_rep_3c02d74,Sales Rep ID_rep_3c17971,Sales Rep ID_rep_3eec4d1,Sales Rep ID_rep_4324a3d,Sales Rep ID_rep_442171a,Sales Rep ID_rep_4477cba,Sales Rep ID_rep_452a29c,Sales Rep ID_rep_45f5ba6,Sales Rep ID_rep_469891d,Sales Rep ID_rep_47b7e08,Sales Rep ID_rep_488c4bf,Sales Rep ID_rep_493273f,Sales Rep ID_rep_49d39a4,Sales Rep ID_rep_4a7474a,Sales Rep ID_rep_4c1407b,Sales Rep ID_rep_4cac043,Sales Rep ID_rep_4d5616b,Sales Rep ID_rep_4fbad23,Sales Rep ID_rep_4fcd9d8,Sales Rep ID_rep_523f514,Sales Rep ID_rep_52a8010,Sales Rep ID_rep_52c3eb3,Sales Rep ID_rep_536a60e,Sales Rep ID_rep_53c4626,Sales Rep ID_rep_567e972,Sales Rep ID_rep_584d289,Sales Rep ID_rep_58988cb,Sales Rep ID_rep_58f4498,Sales Rep ID_rep_5ad3489,Sales Rep ID_rep_5af5623,Sales Rep ID_rep_5b975b2,Sales Rep ID_rep_5bdd867,Sales Rep ID_rep_5bed9c7,Sales Rep ID_rep_5c8b2b8,Sales Rep ID_rep_5d21642,Sales Rep ID_rep_5d420f5,Sales Rep ID_rep_5f9b0b5,Sales Rep ID_rep_620efdc,Sales Rep ID_rep_627b9bb,Sales Rep ID_rep_6372974,Sales Rep ID_rep_639f828,Sales Rep ID_rep_652d327,Sales Rep ID_rep_6705ebb,Sales Rep ID_rep_684c2e1,Sales Rep ID_rep_6873b3c,Sales Rep ID_rep_6951398,Sales Rep ID_rep_69c4542,Sales Rep ID_rep_6a69c35,Sales Rep ID_rep_6a817aa,Sales Rep ID_rep_6a87d28,Sales Rep ID_rep_6ad6615,Sales Rep ID_rep_6cdce0d,Sales Rep ID_rep_6e553b7,Sales Rep ID_rep_6f2b653,Sales Rep ID_rep_6fc14f7,Sales Rep ID_rep_70630bf,Sales Rep ID_rep_7084626,Sales Rep ID_rep_7089803,Sales Rep ID_rep_71cd7dc,Sales Rep ID_rep_720c136,Sales Rep ID_rep_72c1303,Sales Rep ID_rep_74a4d9c,Sales Rep ID_rep_74edb95,Sales Rep ID_rep_74f24b9,Sales Rep ID_rep_76f5957,Sales Rep ID_rep_7740707,Sales Rep ID_rep_785b538,Sales Rep ID_rep_7b00993,Sales Rep ID_rep_7bc1730,Sales Rep ID_rep_7c5f2d7,Sales Rep ID_rep_7e853a4,Sales Rep ID_rep_82bc742,Sales Rep ID_rep_83cfd47,Sales Rep ID_rep_83ec8cf,Sales Rep ID_rep_852d43c,Sales Rep ID_rep_85503ea,Sales Rep ID_rep_85910c4,Sales

In [77]:
df_phase5.dtypes

age                               double[pyarrow]
revenue                                   float64
pushes                            double[pyarrow]
Won                                uint8[pyarrow]
quarter_created                   double[pyarrow]
                                       ...       
hosting_location_on_prem           uint8[pyarrow]
hosting_location_saas_platform     uint8[pyarrow]
sales_territory_Central            uint8[pyarrow]
sales_territory_East               uint8[pyarrow]
sales_territory_West               uint8[pyarrow]
Length: 323, dtype: object

In [78]:
df_phase5.dtypes.value_counts()

uint8[pyarrow]     318
double[pyarrow]      4
float64              1
Name: count, dtype: int64

In [79]:
# importlib.reload(saleslib)
# saleslib.get_stratified_sample(df_phase5, 0.01)

# Output to new file

In [80]:
output_filepath = os.path.splitext(input_filepath)[0] + "_prepped.csv"

df_phase5.to_csv(output_filepath, index=False)
print(f"Data saved to new CSV file:\n{os.path.abspath(output_filepath)}")

Data saved to new CSV file:
/Users/the-molecular-man/source_code/portfolio-private/sales_prediction_modeling/data/raw_CRM_opps_export-dummydata_prepped.csv
